In [1]:
import caffe
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os
from PIL import ImageFont, ImageDraw, Image
import shutil

In [2]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [3]:
input_dir = u'C:\\Users\\Mikhail\\Diploma\\Hieroglyphs1_4\\simple_set_good\\'
output_dir = u'C:\\Users\\Mikhail\\Diploma\\Hieroglyphs1_4\\simple_set_middle\\'
cropped_dir = u'C:\\Users\\Mikhail\\Diploma\\Hieroglyphs1_4\\simple_set_fonts\\'

font_names = []
font_files = [u'simhei.ttf', u'simsun.ttc', u'msjhbd.ttc', u'msjhl.ttc', u'msjh.ttc', u'msjhbd.ttc', u'msjhl.ttc', u'msjh.ttc', u'msyhbd.ttc', u'msyhl.ttc', u'msyh.ttc', u'msyhbd.ttc', u'msyhl.ttc', u'msyh.ttc', u'mingliu.ttc', u'mingliu.ttc', u'simfang.ttf']
for font_file in font_files:
    font_names.append(font_file[:font_file.find('.')])

image_mode = 'L'

size = 46

print font_names

In [4]:
def prepare_dir(dir_name, subdir_name):
    if os.path.exists(dir_name + subdir_name):
        shutil.rmtree(dir_name + subdir_name)
    
    os.makedirs(dir_name + subdir_name)

In [5]:
def draw_hieroglyph(font, filename, subdir):
    hieroglyph = Image.new(image_mode, (size, int(1.3 * size)), color=255)
    draw = ImageDraw.Draw(hieroglyph)
    draw.text((0, 0), filename[0], font=font, fill=0)
    
    hieroglyph.save(output_dir + subdir + '\\' + filename + '.bmp', 'BMP')

In [9]:
fonts = []

for index in range(len(font_names)):
    prepare_dir(output_dir, font_names[index])
    fonts.append(ImageFont.truetype(font_files[index], size))

for filename in os.listdir(input_dir):
    for index in range(len(font_names)):
        draw_hieroglyph(fonts[index], filename, font_names[index])

Обрежем края на шрифтовых изображениях иероглифов

In [6]:
white_pixel = 255

def has_row_black(image, row):
    height, width = image.shape[:2]
    for x in range(0, width):
        if image[row, x] != white_pixel:
            return True
    return False    
    
def has_col_black(image, col):
    height, width = image.shape[:2]
    for y in range(0, height):
        if image[y, col] != white_pixel:
            return True
    return False

def crop_image(image):
    height, width = image.shape[:2]
    
    left = 0
    upper = 0
    right = 0
    lower = 0
    
    for y in range(0, height):
        if has_row_black(image, y):
            upper = y
            break
        
    for x in range(0, width):
        if has_col_black(image, x):
            left = x
            break
    
    for x in range(width - 1, -1, -1):
        if has_col_black(image, x):
            right = x
            break
        
    for y in range(height - 1, -1, -1):
        if has_row_black(image, y):
            lower = y
            break
                
    return image[upper : lower + 1, left : right + 1]

In [16]:
input_dir_2 = u'C:\\Users\\Mikhail\\Diploma\\Hieroglyphs1_4\\simple_set_middle\\'
output_dir_2 = u'C:\\Users\\Mikhail\\Diploma\\Hieroglyphs1_4\\simple_set_fonts_inter_area\\'
default_size = 23

In [17]:
def centre_images(input_dir, output_dir):
    
    iteration = 0

    for filename in os.listdir(input_dir):
        
        #PIL Image
        image = Image.open(input_dir + filename)
        
        #OpenCV image ~ numpy.ndarray
        image = np.array(image)
        
        cropped_image = crop_image(image)
        
        image_background = np.zeros((default_size,default_size), np.uint8)
        image_background[:] = (255)
        
        height, width = cropped_image.shape[:2]
        new_width = default_size
        new_height = default_size

        left = 0
        upper = 0

        if width >= height:
            new_height = int(round(float(height) / width * default_size))
            upper = int((default_size - new_height) / 2.0)
        else:
            new_width = int(round(float(width) / height * default_size))
            left = int((default_size - new_width) / 2.0)
        
        resized_image = cv2.resize(cropped_image, (new_width, new_height), interpolation = cv2.INTER_AREA)
        
        image_background[upper : upper + new_height, left : left + new_width] = resized_image
        
        pil_image_background = Image.fromarray(image_background)
        
        pil_image_background.save(output_dir + filename, 'BMP')

In [18]:
for index in range(len(font_names)):
    if not os.path.exists(output_dir_2 + font_names[index]):
        prepare_dir(output_dir_2, font_names[index])
        centre_images(input_dir_2 + font_names[index] + '\\', output_dir_2 + font_names[index] + '\\')

Добавим гауссовский шум

In [9]:
import random

In [10]:
def add_gauss_noise(image, std):
    width, height = image.size
    for i in range(0, width):
        for j in range(0, height):
            random_addition = random.gauss(0, std)
            image.putpixel((i, j), (int(min(255, max(0, image.getpixel((i,j)) + random_addition)))))

In [20]:
def gauss_images(input_dir, output_dir, std, x_product):
    for filename in os.listdir(input_dir):
        for index in range(x_product):
            image = Image.open(input_dir + filename)
            add_gauss_noise(image, std)
            image.save(output_dir + filename[:filename.find('.')] + str(index) + '.bmp', 'BMP')

In [21]:
input_dir_gauss = u'C:\\Users\\Mikhail\\Diploma\\Hieroglyphs2\\simple_set_fonts\\'
output_dir_gauss = u'C:\\Users\\Mikhail\\Diploma\\Hieroglyphs2\\simple_set_fonts_gauss_'

stds = [1,2,3,4,5]
x_products = [20,20,20,20,20]

for std_index in range(len(stds)):
    for index in range(len(font_names)):
        output_dir_gauss_spec = output_dir_gauss + str(stds[std_index]) + '\\'
        if not os.path.exists(output_dir_gauss_spec + font_names[index]):
            prepare_dir(output_dir_gauss_spec, font_names[index])
            gauss_images(input_dir_gauss + font_names[index] + '\\', output_dir_gauss_spec + font_names[index] + '\\', stds[std_index], x_products[std_index])